In [2]:
import pandas as pd
import numpy as np
import glob

files = glob.glob("../data/raw/data_Q3_2025/*.csv")[:10]

important_smarts = [
    "smart_5_raw",    # Reallocated sectors
    "smart_9_raw",    # Power-on hours
    "smart_187_raw",  # Reported uncorrectable
    "smart_188_raw",  # Command timeout
    "smart_197_raw",  # Pending sectors
    "smart_198_raw"   # Offline uncorrectable
]

use_cols = [
    "date",
    "serial_number",
    "model",
    "capacity_bytes",
    "failure"
] + important_smarts

df = pd.concat(
    (pd.read_csv(f, usecols=use_cols) for f in files),
    ignore_index=True
)

In [3]:
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(["serial_number", "date"])

In [4]:
df = df.drop(columns=[col for col in df.columns if "fail_date" in col], errors="ignore")

fail_dates = (
    df.loc[df["failure"] == 1]
      .groupby("serial_number")["date"]
      .min()
      .rename("fail_date")
)
df = df.merge(fail_dates, on="serial_number", how="left")

In [5]:
window = 7

df["fail_in_7_days"] = (
    (df["fail_date"].notna()) &
    (df["date"] < df["fail_date"]) &
    (df["date"] >= df["fail_date"] - pd.Timedelta(days=window))
).astype(int)

df_train = df[
    (df["fail_date"].isna()) |
    (df["date"] <= df["fail_date"])
]

In [6]:
df.head()

,date,serial_number,model,capacity_bytes,failure,smart_5_raw,smart_9_raw,smart_187_raw,smart_188_raw,smart_197_raw,smart_198_raw,fail_date,fail_in_7_days
0,2025-07-01,000a43e7dee60010,DELLBOSS VD,480036847616,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
1,2025-07-02,000a43e7dee60010,DELLBOSS VD,480036847616,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
2,2025-07-03,000a43e7dee60010,DELLBOSS VD,480036847616,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
3,2025-07-04,000a43e7dee60010,DELLBOSS VD,480036847616,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
4,2025-07-05,000a43e7dee60010,DELLBOSS VD,480036847616,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0


In [7]:
df["fail_in_7_days"].value_counts()

fail_in_7_days
0    3223219
1        374
Name: count, dtype: int64

In [8]:
df.groupby("serial_number")["fail_in_7_days"].sum().describe()

count    322690.000000
mean          0.001159
std           0.078205
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.000000
Name: fail_in_7_days, dtype: float64

In [9]:
df[df["failure"] == 1]["serial_number"].nunique()

106

In [10]:
print(df["failure"].value_counts())

failure
0    3223487
1        106
Name: count, dtype: int64


In [11]:
import os

os.makedirs("../data/processed", exist_ok=True)
df.to_parquet("../data/processed/feature_engineered.parquet", index=False)